# Review process for generating steady state conditions

In [2]:
# Specify quail source directory
# Import standard libraries
import matplotlib.pyplot as plt
import numpy as np
import os

# Modify base path for depending on your file structure.
BASE_PATH = "/Users/paxton/git"

# Specify path for Quail source code
source_dir = f"{BASE_PATH}/quail_volcano/src"

# Import quail modules
os.chdir(source_dir)

# Import steady_state module
import compressible_conduit_steady.steady_state as steady_state



In [ ]:
R = 5
f_plug = 1.9e8
len_plug = 50

t_plug = f_plug/(2*np.pi*R*len_plug)
trac_par = 2*t_plug/R

#print(trac_par, t_plug/1e5, f_plug/(2*np.pi*R*1000))
# Construct x_mesh (with shape (n, 1, 1))

N_mesh_points = 800

x_mesh = np.linspace(-1000, 0, N_mesh_points)[:,np.newaxis,np.newaxis]
# Chamber pressure is not used, so we pass a dummy value
p_chamber = None
# Set vent pressure

p_vent = (1e5)

# Set functions for traction, total water mass fraction, crystal mass fraction, temperature
# Define the cosine taper function
def cosine_taper(x, x1, x2, y1, y2):
    return np.where(x < x1, y1,
                    np.where(x > x2, y2,
                             y1 + (y2 - y1) * 0.5 * (1 - np.cos(np.pi * (x - x1) / (x2 - x1)))))

# Define the transition region
x1 = -len_plug - 10  # Start of transition
x2 = -len_plug + 10  # End of transition

T_chamber = 950 + 273.15
yC = 0.4
yWt = 0.006

# Define the functions using cosine taper
traction_fn = lambda x: cosine_taper(x, x1, x2, 0, -trac_par)
yWt_fn = lambda x: cosine_taper(x, x1, x2,yWt, 0.)
yC_fn = lambda x: cosine_taper(x, x1, x2, yC, 0.95)
T_fn = lambda x: cosine_taper(x, x1, x2, T_chamber, 930 + 273.15)
yF_fn = lambda x: cosine_taper(x, x1, x2, 0, 1)

# Set material properties of the magma phase (melt + dissolved water + crystals)
material_props = {
  "yA": 1e-7,          # Air mass fraction (> 0 for numerics)
  "c_v_magma": 1e3,    # Magma phase heat capacity per mass
  "rho0_magma": 2.6e3, # Linearization reference density
  "K_magma": 1e10,     # Bulk modulus
  "p0_magma": 36e6,    # Linearization reference pressure
  "solubility_k": 2.8e-6, # Henry's law coefficient
  "solubility_n": 0.5, # Henry's law exponent
}

# Initialize hydrostatic steady-state solver
# This is a one-use callable object
f = steady_state.StaticPlug(x_mesh,
                            p_chamber,
                            traction_fn, yWt_fn, yC_fn, T_fn, yF_fn,
                            override_properties=material_props, enforce_p_vent=p_vent)
# Solve by calling f
#   io_format="p" here returns only pressure
#   io_format="quail" will return the solution in quail format
p = f(x_mesh, is_solve_direction_downward=True, io_format="p")

# Solve again in Quail format (need to reinitialize f)
f = steady_state.StaticPlug(x_mesh,
                            p_chamber,
                            traction_fn, yWt_fn, yC_fn, T_fn, yF_fn,
                            override_properties=material_props, enforce_p_vent=p_vent)

U = f(x_mesh, is_solve_direction_downward=True, io_format="quail")

48383.102699936186 1.2095775674984046 6047.887837492023


## Relevant outputs from steady state solution

In [ ]:
print(f"Chamber pressure: {p[0,0]} [Pa]")


Chamber pressure: 25174368.352108344 [Pa]


In [16]:
U.shape
U[0,:,:]

array([[2.59717619e-04, 0.00000000e+00, 2.59717593e+03, 0.00000000e+00,
        3.17673597e+09, 1.55830571e+01, 1.03887048e+03, 0.00000000e+00]])